# 데이터 드리프트 모니터링

시간이 지남에 따라 기능 데이터의 추세 변화로 인해 모델의 예측 정확도가 떨어질 수 있습니다. 이러한 현상을 *데이터 드리프트*라고 합니다. 필요한 경우 모델을 재학습시킬 수 있도록 기계 학습 솔루션을 모니터링하여 데이터 드리프트를 검색해야 합니다.

이 랩에서는 데이터 세트의 데이터 드래프트 모니터링을 구성합니다.

## 시작하기 전 주의 사항

이 Notebook의 코드를 실행하려면 최신 버전의 **azureml-sdk** 및 **azureml-widgets** 패키지 외에 **azureml-datadrift** 패키지도 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

In [ ]:
!pip show azureml-datadrift

## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## *기준* 데이터 세트 만들기

데이터 세트에서 데이터 드리프트를 모니터링하려면 향후 수집되는 데이터와의 비교 지점으로 사용할 *기준* 데이터 세트(대개 모델을 학습시키는 데 사용한 데이터 세트)를 등록해야 합니다. 

In [ ]:
from azureml.core import Datastore, Dataset


# 기준 데이터 업로드
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-baseline',
                       overwrite=True, 
                       show_progress=True)

# 파일 데이터 세트 만들기 및 등록
print('Registering baseline dataset...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-baseline/*.csv'))
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='diabetes baseline',
                           description='diabetes baseline data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Baseline dataset registered!')

## *대상* 데이터 세트 만들기

기준 학습 데이터와 같은 기능이 포함된 새 데이터를 일정 기간 동안 수집할 수 있습니다. 이 새 데이터를 기준 데이터와 비교하려면 데이터 드리프트를 분석할 기능이 포함된 대상 데이터 세트를 정의해야 하며, 새 데이터가 최신 상태였던 시점을 나타내는 타임스탬프 필드도 정의해야 합니다. 이렇게 하면 여러 시간 간격에서 데이터 드리프트를 측정할 수 있습니다. 타임스탬프는 데이터 세트 자체의 필드일 수도 있고, 데이터를 저장하는 데 사용되는 폴더 및 파일 이름 패턴에서 파생될 수도 있습니다. 예를 들어 특정 연도의 폴더가 들어 있는 폴더 계층 구조에 새 데이터를 저장할 수 있습니다. 이 폴더에는 월별 폴더를 포함하고, 월별 폴더에는 일별 폴더를 포함할 수 있습니다. 또는 *data_2020-01-29.csv*와 같이 파일 이름에 년-월-일을 인코딩할 수도 있습니다. 다음 코드에서 이 두 번째 방식이 사용됩니다.

In [ ]:
import datetime as dt
import pandas as pd

print('Generating simulated data...')

# 두 데이터 파일 중 더 작은 파일 로드
data = pd.read_csv('data/diabetes2.csv')

# 지난 6주 동안의 데이터 생성
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    # Get the date X weeks ago
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # Modify data to ceate some drift
    data['Pregnancies'] = data['Pregnancies'] + 1
    data['Age'] = round(data['Age'] * 1.2).astype(int)
    data['BMI'] = data['BMI'] * 1.1
    
    # Save the file with the date encoded in the filename
    file_path = 'data/diabetes_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# 파일 업로드
path_on_datastore = 'diabetes-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# 폴더 파티션 형식을 사용하여 'date' 타임스탬프 열이 있는 데이터 세트 정의
partition_format = path_on_datastore + '/diabetes_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# 대상 데이터 세트 등록
print('Registering target dataset...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='diabetes target',
                                                                          description='diabetes target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Target dataset registered!')

## 데이터 드리프트 모니터 만들기

이제 당뇨병 데이터용 데이터 드리프트 모니터를 만들 수 있습니다. 데이터 드리프트 모니터는 주기적으로 실행되거나 요청 시에 실행되어 기준 데이터 세트를 대상 데이터 세트와 비교합니다. 시간이 지남에 따라 대상 데이터 세트에는 새 데이터가 추가됩니다.

### 컴퓨팅 대상 만들기

데이터 드리프트 모니터를 실행하려면 컴퓨팅 대상이 필요합니다. 다음 셀의 명령을 실행하여 컴퓨팅 클러스터를 지정합니다. 이 클러스터는 없으면 생성됩니다.

> **중요**: 컴퓨팅 클러스터를 실행하기 전에 아래 코드에서 *your-compute-cluster*를 컴퓨팅 클러스터의 이름으로 변경하세요! 클러스터 이름은 2~16자 사이의 전역으로 고유한 이름이어야 합니다. 유효한 문자는 영문자, 숫자 및 문자입니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **참고**: 컴퓨팅 인스턴스와 클러스터는 표준 Azure 가상 머신 이미지를 기반으로 합니다. 이 연습에서는 비용과 성능 간 최적의 균형을 달성하기 위해 *Standard_DS11_v2* 이미지를 사용하는 것이 좋습니다. 구독의 할당량이 적어 이 이미지를 포함할 수 없는 경우 대체 이미지를 선택할 수 있습니다. 그러나 큰 이미지는 높은 비용을 야기할 수 있고 작은 이미지는 작업을 완료하는 데 충분하지 않을 수 있으므로 신중히 선택하는 것이 좋습니다. Azure 관리자에게 요청하여 할당량을 늘릴 수도 있습니다.

### 데이터 드리프트 모니터 정의

이제 **DataDriftDetector** 클래스를 사용하여 데이터에 대한 데이터 드리프트 모니터를 정의할 준비가 되었습니다. 데이터 드리프트를 모니터링할 기능, 모니터링 프로세스를 실행하는 데 사용할 컴퓨팅 대상의 이름, 데이터를 비교할 빈도, 초과되는 경우 경고를 트리거할 데이터 드리프트 임계값, 그리고 데이터 수집에서 허용할 대기 시간(시간 단위)을 지정할 수 있습니다.

In [ ]:
from azureml.datadrift import DataDriftDetector

# 기능 목록 설정
features = ['Pregnancies', 'Age', 'BMI']

# 데이터 드리프트 탐지기 설정
monitor = DataDriftDetector.create_from_datasets(ws, 'mslearn-diabates-drift', baseline_data_set, target_data_set,
                                                      compute_target=cluster_name, 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

## 데이터 드리프트 모니터 백필

이제 6주 동안 시뮬레이션된 주간 데이터 수집을 통해 수집된 데이터를 포함하는 기준 데이터 세트와 대상 데이터 세트가 생성되었습니다. 이 두 데이터 세트를 사용하여 모니터를 백필하면 모니터가 원래 기준 데이터와 대상 데이터 간의 데이터 드리프트를 분석할 수 있습니다.

> **참고** 백필 분석을 실행하려면 컴퓨팅 대상을 시작해야 하므로, 백필 실행은 시간이 다소 걸릴 수 있습니다. 위젯이 업데이트되어 실험 상태를 표시하지 않는 경우도 있으므로 링크를 클릭하여 Azure Machine Learning Studio에서 실험 상태를 관찰하세요.

In [ ]:
from azureml.widgets import RunDetails

backfill = monitor.backfill(dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

## 데이터 드리프트 분석

다음 코드를 사용하면 백필 실행에서 수집된 시점의 데이터 드리프트를 검사할 수 있습니다.

In [ ]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

다음 단계를 수행하여 [Azure Machine Learning Studio](https://ml.azure.com)에서 데이터 드리프트 메트릭을 시각화할 수도 있습니다.

1. **데이터 세트** 페이지에서 **데이터 세트 모니터** 탭을 표시합니다.
2. 확인할 데이터 드리프트 모니터를 클릭합니다.
3. 데이터 드리프트 메트릭을 확인할 날짜 범위를 선택합니다. 세로 막대형 차트에 여러 주의 데이터가 표시되지 않으면 1분 정도 기다렸다가 **새로 고침**을 클릭하세요.
4. 위쪽의 **드리프트 개요** 섹션에 표시되는 차트를 살펴봅니다. 이러한 차트에는 전반적인 드리프트 크기와 기능당 드리프트의 영향 정도가 표시됩니다.
5. 아래쪽의 **기능 세부 정보** 섹션에 표시되는 차트를 살펴봅니다. 그러면 개별 기능에서 측정된 여러 드리프트 값을 확인할 수 있습니다.

> **참고**: 데이터 드리프트 메트릭을 파악하는 데 도움이 되는 정보를 확인하려면 Azure Machine Learning 설명서에서 [데이터 세트를 모니터링하는 방법](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets#understanding-data-drift-results)을 참조하세요.

## 추가 학습

이 랩의 목표는 데이터 드리프트 모니터링의 개념과 원칙을 소개하는 것입니다. 데이터 세트를 사용하여 데이터 드리프트를 모니터링하는 방법에 대한 자세한 내용은 Azure Machine Learning 설명서에서 [데이터 세트에서 데이터 드리프트 검색](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets)을 참조하세요.

게시된 서비스에서 데이터를 수집한 다음 데이터 드리프트 모니터링용 대상 데이터 세트로 사용할 수도 있습니다. 자세한 내용은 [프로덕션 환경에서 모델의 데이터 수집](https://docs.microsoft.com/azure/machine-learning/how-to-enable-data-collection)을 참조하세요.
